<span style='font-size:xx-large'>**Code Exportation Polymorphic Graph & Serialization**</span>



In [2]:
import tensorflow as tf

model = tf.saved_model.load("saved_model")

def preprocess_audio(audio_input, sample_rate=16000, segment_duration=1, frame_duration=0.025,
                     hop_duration=0.010, num_bands=50):
    segment_samples = int(segment_duration * sample_rate)
    frame_samples = int(frame_duration * sample_rate)
    hop_samples = int(hop_duration * sample_rate)

    audio_data_padded = tf.pad(tf.squeeze(audio_input),
                               [[0, tf.maximum(0, segment_samples - tf.shape(audio_input)[0])]])

    audio_data_normalized = tf.math.l2_normalize(audio_data_padded)

    stft = tf.signal.stft(audio_data_normalized, fft_length=512, frame_length=frame_samples, frame_step=hop_samples)
    magnitude_spectrogram = tf.abs(stft)
    mel_filterbank = tf.signal.linear_to_mel_weight_matrix(num_bands, tf.shape(magnitude_spectrogram)[-1],
                                                           sample_rate)
    mel_spectrogram = tf.matmul(magnitude_spectrogram, mel_filterbank)

    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, axis=-1)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, axis=0)

    return log_mel_spectrogram


class ExportModel(tf.Module):

    def __init__(self, model):
        super().__init__()
        self.model = model

        self.call_string_input = self.__call__.get_concrete_function(
            input=tf.TensorSpec(shape=(), dtype=tf.string))
        self.call_float_input = self.__call__.get_concrete_function(
            input=tf.TensorSpec(shape=[None, 16000], dtype=tf.float32))

    @tf.function
    def __call__(self, input):
        class_labels = ["background", "down", "go", "left", "no", "off", "on", "right", "stop", "up", "yes", "unknown"]
        if input.dtype == tf.string:
            input = tf.io.read_file(input)
            input, _ = tf.audio.decode_wav(input, desired_channels=1, desired_samples=16000, )
            input = tf.squeeze(input, axis=-1)

        input = preprocess_audio(input)
        infer = model.signatures["serving_default"]
        result = infer(input)['output_0']

        class_ids = tf.argmax(result, axis=-1)
        class_names = tf.gather(class_labels, class_ids)

        return {'predictions': result,
                'class_ids': class_ids,
                'class_names': class_names}


export = ExportModel(model)

tf.saved_model.save(export, "SavedModelCNN", signatures={
    'serving_default': export.call_string_input,
    'serving_float': export.call_float_input})

loaded = tf.saved_model.load("SavedModelCNN")
infer = loaded.signatures["serving_default"]
print(list(loaded.signatures.keys()))

# Batch Waveform:
x = '0c40e715_nohash_1.wav'
x = tf.io.read_file(x)
x, _ = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000, )
x = tf.squeeze(x, axis=-1)
output = loaded.signatures['serving_float'](x)
print(output)
# String.wav File:
out = loaded("0c40e715_nohash_1.wav")
print(out)


2024-05-09 18:29:38.191645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 18:29:41.274839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OSError: No file or directory found at qPrunedCNNLSTM.keras

<span style='font-size:xx-large'>Graph to TensorFlow Lite</span>


In [0]:
import tensorflow as tf


model = tf.saved_model.load("SavedModelGraph")

converter = tf.lite.TFLiteConverter.from_saved_model("SavedModelGraph")
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS,
  tf.lite.OpsSet.SELECT_TF_OPS
]

tflite_model = converter.convert()


with open('saved_model_optimized.tflite', 'wb') as f:
  f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_path="saved_model_optimized.tflite")
signatures = interpreter.get_signature_list()
print(signatures)

<span style='font-size:xx-large'>Signature Serving & Inference Using Optimized TensorFlow Model</span>


In [0]:
import tensorflow as tf

# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(model_path="cnn_optimized.tflite")

# Print the signatures from the converted model
signatures = interpreter.get_signature_list()
print('Signature:', signatures)

# string and waveform are callable with input as arguments.
string = interpreter.get_signature_runner('serving_default')
waveform = interpreter.get_signature_runner('serving_float')


x = '0c40e715_nohash_1.wav'
x = tf.io.read_file(x)
x, _ = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000,)
x = tf.squeeze(x, axis=-1)

output = waveform(input=x)
print('Predictions:', output)
